# Get the Raw Data by using API

In [ ]:
# to call the OpenChargeMap API
import requests
# to save the fetched data
import json
# to control request rate and calculate runtime
import time
# to generate grid of lat/lon coordinates
import numpy as np
# Optional: silence warnings
import urllib3



# === CONFIG ===
API_KEY = "your_api_key_here"  # Replace with your Open Charge Map API key
BASE_URL = "https://api.openchargemap.io/v3/poi/"
OUTPUT_FILE = "../data/ocm_germany_full.json"
SLEEP_BETWEEN_REQUESTS = 1
MAX_RESULTS = 1000
DISTANCE_KM = 50

# === DEV MODE ===
DEV_MODE = False
DEV_LIMIT = 5  # Max number of stations to fetch in dev mode

# === Generate Grid Over Germany ===
lat_range = np.arange(47.0, 55.5, 1.0)
lon_range = np.arange(5.5, 15.5, 1.0)
regions = [(round(lat, 2), round(lon, 2)) for lat in lat_range for lon in lon_range]

fetched_ids = set()
all_data = []
start_time = time.time()

print(f"🚀 Fetching Open Charge Map data for Germany using {len(regions)} tiles...\n")

for i, (lat, lon) in enumerate(regions, start=1):
    print(f"\n📍 Region {i}/{len(regions)}: lat={lat}, lon={lon}")
    offset = 0

    while True:
        params = {
            "output": "json",
            "latitude": lat,
            "longitude": lon,
            "distance": DISTANCE_KM,
            "distanceunit": "KM",
            "maxresults": MAX_RESULTS,
            "offset": offset,
            "sort": "ID",
            "compact": "false",
            "key": API_KEY
        }

        response = requests.get(BASE_URL, params=params, verify=False)

        if response.status_code != 200:
            print("❌ API error:", response.status_code, response.text)
            break

        batch = response.json()
        if not batch:
            print("✅ No more results in this region.")
            break

        new_records = 0
        for entry in batch:
            station_id = entry.get("ID")
            if station_id and station_id not in fetched_ids:
                fetched_ids.add(station_id)
                all_data.append(entry)
                new_records += 1

                if DEV_MODE and len(all_data) >= DEV_LIMIT:
                    print(f"🧪 DEV Mode: Reached {DEV_LIMIT} records.")
                    break  # Exit entry loop

        print(f"➕ Added {new_records} new records from offset {offset}")

        if new_records == 0 or (DEV_MODE and len(all_data) >= DEV_LIMIT):
            print("⛔ Stopping pagination for this region.")
            break  # Exit pagination loop

        offset += MAX_RESULTS
        time.sleep(SLEEP_BETWEEN_REQUESTS)

    if DEV_MODE and len(all_data) >= DEV_LIMIT:
        print("✅ DEV Mode: Stopping outer loop early.")
        break  # Exit region loop

# === Save Results ===
with open(OUTPUT_FILE, "w", encoding="utf-8") as f:
    json.dump(all_data, f, indent=4, ensure_ascii=False)

runtime = round(time.time() - start_time, 2)
print(f"\n🎉 Finished! Total unique records: {len(all_data)}")
print(f"📁 Data saved to: {OUTPUT_FILE}")
print(f"⏱️ Runtime: {runtime} seconds")


# Export Data in csv format for SQL-Database

In [ ]:
import json
import pandas as pd

# 🔁 Load your JSON data (replace with your actual file)
with open("../data/ocm_germany_full.json", "r", encoding="utf-8") as f:
    data = json.load(f)

# 🧱 Setup data containers
stations, connections = [], []
operators, usage_types, status_types = {}, {}, {}
connection_types, charging_levels, current_types = {}, {}, {}
data_providers, countries = {}, {}

for item in data:
    station_id = item["ID"]
    address = item.get("AddressInfo", {})
    country = address.get("Country", {})

    stations.append({
        "station_id": station_id,
        "title": address.get("Title"),
        "city": address.get("Town"),
        "postcode": address.get("Postcode"),
        "country_id": country.get("ID"),
        "latitude": address.get("Latitude"),
        "longitude": address.get("Longitude"),
        "operator_id": item.get("OperatorID"),
        "usage_type_id": item.get("UsageTypeID"),
        "status_type_id": item.get("StatusTypeID"),
        "data_provider_id": item.get("DataProviderID"),
        "date_created": item.get("DateCreated"),
        "date_updated": item.get("DateLastStatusUpdate")
    })

    op = item.get("OperatorInfo")
    if op:
        operators[op["ID"]] = {
            "operator_id": op["ID"],
            "title": op.get("Title"),
            "website": op.get("WebsiteURL"),
        }

    if country:
        countries[country["ID"]] = {
            "country_id": country["ID"],
            "title": country.get("Title"),
            "iso_code": country.get("ISOCode"),
            "continent_code": country.get("ContinentCode")
        }

    for conn in item.get("Connections", []):
        connections.append({
            "connection_id": conn.get("ID"),
            "station_id": station_id,
            "connection_type_id": conn.get("ConnectionTypeID"),
            "level_id": conn.get("LevelID"),
            "current_type_id": conn.get("CurrentTypeID"),
            "voltage": conn.get("Voltage"),
            "amps": conn.get("Amps"),
            "power_kw": conn.get("PowerKW"),
            "quantity": conn.get("Quantity"),
        })

        ct = conn.get("ConnectionType")
        if ct:
            connection_types[ct["ID"]] = {
                "connection_type_id": ct["ID"],
                "title": ct.get("Title"),
                "formal_name": ct.get("FormalName"),
                "is_obsolete": ct.get("IsObsolete"),
                "is_discontinued": ct.get("IsDiscontinued")
            }

        curr = conn.get("CurrentType")
        if curr:
            current_types[curr["ID"]] = {
                "current_type_id": curr["ID"],
                "title": curr.get("Title"),
                "description": curr.get("Description")
            }


df_stations = pd.DataFrame(stations)
df_stations = df_stations.astype({
    "station_id": "Int64",
    "title": "string",
    "city": "string",
    "postcode": "string",   # safer than float, to keep leading 0s
    "country_id": "Int64",
    "latitude": "float64",
    "longitude": "float64",
    "operator_id": "Int64",
    "usage_type_id": "Int64",
    "status_type_id": "Int64",
    "data_provider_id": "Int64",
    "date_created": "string",  # parse as datetime separately
    "date_updated": "string"
})
df_stations.to_csv("../data/charging_stations.csv", index=False)

df_connections = pd.DataFrame(connections)

# Round float values before casting to Int64
for col in [
    "connection_id", "station_id", "connection_type_id",
    "level_id", "current_type_id", "voltage", "amps",
    "power_kw", "quantity"
]:
    if col in df_connections.columns:
        df_connections[col] = df_connections[col].apply(
            lambda x: round(x) if pd.notnull(x) else pd.NA
        )

df_connections = df_connections.astype({
    "connection_id": "Int64",
    "station_id": "Int64",
    "connection_type_id": "Int64",
    "level_id": "Int64",
    "current_type_id": "Int64",
    "voltage": "Int64",
    "amps": "Int64",
    "power_kw": "Int64",
    "quantity": "Int64"
})
df_connections.to_csv("../data/connections.csv", index=False)

df_operators = pd.DataFrame(operators.values())
df_operators = df_operators.astype({
    "operator_id": "Int64",
    "title": "string",
    "website": "string"
})
df_operators.to_csv("../data/operators.csv", index=False)

df_countries = pd.DataFrame(countries.values())
df_countries = df_countries.astype({
    "country_id": "Int64",
    "title": "string",
    "iso_code": "string",
    "continent_code": "string"
})
df_countries.to_csv("../data/countries.csv", index=False)

df_current_types = pd.DataFrame(current_types.values())
df_current_types = df_current_types.astype({
    "current_type_id": "Int64",
    "title": "string",
    "description": "string"
})
df_current_types.to_csv("../data/current_types.csv", index=False)

df_connection_types = pd.DataFrame(connection_types.values()).astype({
    "connection_type_id": "Int64",
    "title": "string",
    "formal_name": "string",
    "is_obsolete": "boolean",
    "is_discontinued": "boolean"
})
df_connection_types.to_csv("../data/connection_types.csv", index=False)


print(f"Exported {len(stations)} stations")
print(f"Exported {len(connections)} connections")
print(f"Exported {len(operators)} operators")
print(f"Exported {len(countries)} countries")
print(f"Exported {len(current_types)} current types")
print(f"Exported {len(connection_types)} connection types")

